In [1]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#install fast ai
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [0]:
#import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [4]:
#get biopaper data 
trainCsvPath = Path('drive/My Drive/Year5/csc4006 Final Project/data/bioPaperExport_text_label.csv')
preDf = pd.read_csv(trainCsvPath)
preDf.shape

(29319, 2)

In [0]:
textPath = 'drive/My Drive/Year5/csc4006 Final Project/data/'
textCsv = 'bioPaperExport_text_label.csv'

In [6]:
#Split each abstract into sentences
df = pd.DataFrame(columns=['text','label'])
sList = []
for i,row in preDf.iterrows():
  text = row['text']
  label = row['label']
  sent = text.split('. ')
  sentences = [s.strip() for s in sent]
  sentences = list(filter(None,sentences))
  for s in sentences:
    sList.append((label,s))
    
#sList = ('label','text') + sList
#print(sList[:2])
df = pd.DataFrame(sList,columns=['label','text'])
df.shape
  
  
  


(257739, 2)

In [7]:
#using nltk to split by sentences
#use punkt sentence tokenizer

import nltk
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

ab01 = 'The emergence of multicellularity requires the coexistence of diverse cellular populations displaying cooperative relationships. This enables long-term persistence of cellular consortia, particularly under environmental constraints that challenge cell survival. Toxic environments are known to trigger the formation of multicellular consortia capable of dealing with waste while promoting cell diversity as a way to overcome selection barriers. In this context, recent theoretical studies suggest that an environment containing both resources and toxic waste can promote the emergence of complex, spatially distributed proto-organisms exhibiting division of labor and higher-scale features beyond the cell-cell pairwise interactions. Some previous non-spatial models suggest that the presence of a growth inhibitor can trigger the coexistence of competitive species in an antibiotic-resistance context. In this paper we further explore this idea using both mathematical and computational models taking the most fundamental features of the proto-organisms interactions. It is shown that this resource-waste environmental context, in which both species are lethally affected by the toxic waste and metabolic tradeoffs are present, favours the maintenance of diverse populations. A spatial, stochastic extension confirms our basic results. The evolutionary and ecological implications of these results are outlined.'
sentences = tokenizer.tokenize(ab01)
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['The emergence of multicellularity requires the coexistence of diverse cellular populations displaying cooperative relationships.', 'This enables long-term persistence of cellular consortia, particularly under environmental constraints that challenge cell survival.', 'Toxic environments are known to trigger the formation of multicellular consortia capable of dealing with waste while promoting cell diversity as a way to overcome selection barriers.', 'In this context, recent theoretical studies suggest that an environment containing both resources and toxic waste can promote the emergence of complex, spatially distributed proto-organisms exhibiting division of labor and higher-scale features beyond the cell-cell pairwise interactions.', 'Some previous non-spatial models suggest that the presence of a growth inhibitor can trigger the coexistence of competitive species in an antibiot

In [0]:

df = pd.DataFrame(columns=['text','label'])
sList = []
for i,row in preDf.iterrows():
  text = row['text']
  label = row['label']
  sentences = tokenizer.tokenize(text)
   
  for s in sentences:
    sList.append((label,s))


In [9]:
df = pd.DataFrame(sList,columns=['label','text'])
df.shape

(252971, 2)

In [0]:
df.text.sort_values()

In [10]:
df.label.unique()

array(['neuroscience', 'cancer biology', 'cell biology', 'bioengineering', 'molecular biology', 'biochemistry',
       'developmental biology', 'biophysics', 'systems biology', 'plant biology', 'evolutionary biology', 'zoology',
       'bioinformatics', 'animal behavior and cognition', 'ecology', 'genomics',
       'scientific communication and education', 'genetics', 'immunology', 'pharmacology and toxicology',
       'microbiology', 'physiology', 'epidemiology', 'synthetic biology', 'clinical trials', 'pathology',
       'paleontology', 'unknown'], dtype=object)

In [0]:
from google.colab import files
output = [x[1] for x in sList]
with open('sentences_out.txt', 'w+') as f:
    for item in output:
        f.write("%s\n" % item)

files.download('sentences_out.txt')       

In [11]:

lSent = df.loc[df['text'].str.len() >= 70]
lSent.shape

df = lSent
df.shape


(242838, 2)

In [12]:
#Find average length
df.apply(lambda x: np.mean(x.str.len())).reset_index(name='mean_len_text')

,index,mean_len_text
0,label,13.094825
1,text,167.566625


In [46]:
a = smallSent.shape[0]
b = df.shape[0]

a/b*100


6.686141889781832

In [13]:
#Split data into training/testing sets
from sklearn.model_selection import train_test_split
trainDf, testDf = train_test_split(df,stratify=df['label'], test_size = 0.33, random_state = 11)

trainDf.shape,testDf.shape

((162701, 2), (80137, 2))

In [0]:
#create models from data 
data_lm = TextLMDataBunch.from_df(train_df=trainDf,valid_df=testDf,path='',text_cols='text', label_cols='label')

In [15]:
data_lm.show_batch()

idx,text
0,"passive passenger . xxbos xxmaj cancer metastasis is a process in which a primary tumor spreads to form life - threatening metastases . xxbos xxmaj we confirmed this observation in vivo whereby growth of the xxup c. glabrata xxmaj xxunk strain was permitted only in flies with suppressed production of reactive oxygen species ( xxup ros ) . xxbos xxmaj overall , these findings suggest that stimuli experienced as predictive"
1,"would be useful for understanding the mechanisms of cell fate determination . xxbos xxmaj yet , the literature offers neither a satisfactory definition for ' structured xxup rna regions ' , nor a computational method to accurately identify such regions . xxbos xxmaj besides confirming the prevalence of known tumor associated viruses such as xxup ebv , xxup hbv and several xxup hpv types , numerous novel features were discovered"
2,"changing solvent viscosity , a prediction that is more easily testable in xxup rna than proteins . xxbos xxmaj therefore , controlling xxmaj dma1′s xxup spb dynamics in anaphase is an essential step in xxup s. pombe cell division and the silencing of the xxmaj dma1-dependent mitotic checkpoint . xxbos xxmaj it recapitulates the biology of stromal influence in prostate cancer progression . xxbos xxmaj the results of the present"
3,"synthesis shortly after birth , and is controlled by macromolecular complexes that enhance or suppress transcription by cis - elements scattered throughout the locus . xxbos mitoz is free open source software distributed under gplv3 license and available at https : / / github.com / xxunk / xxunk xxbos xxmaj our analysis shows that several other proteins involved in membrane remodeling at various organelles have similar potential to exploit localization"
4,"populations often occurs through multiple independent mutations responding in parallel to a common selection pressure . xxbos xxmaj this includes steps for processing raw sequence reads including read alignment , and all the way up to variant calling using xxup gatk . xxbos xxmaj we found that dmvs represent extensive portions of seed genomes , range in size from 5 to 76 kb , are scattered throughout all chromosomes ,"


In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [17]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [0]:
learn.recorder.plot(skip_end=15)

In [19]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,4.439908,4.224707,0.321819


In [0]:
#save 
learn.save('bio3_punkt_fit_head')

In [0]:
learn.load('bio3_punkt_fit_head')

In [0]:
learn.unfreeze()

In [24]:
#finetuning
learn.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy
1,4.065968,4.047027,0.336774


epoch,train_loss,valid_loss,accuracy


In [0]:
learn.save('fine_tuned_1_epoch')

In [0]:
learn.load('fine_tuned_1_epoch');

In [0]:
TEXT = "Ryan is a boy"
N_WORDS = 40
N_SENTENCES = 2
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [0]:
data_clas = TextClasDataBunch.from_df(path='',train_df=trainDf, valid_df=testDf, vocab=data_lm.train_ds.vocab,text_cols='text', label_cols='label')

In [0]:
learn.save_encoder('fine_tuned_enc')

In [0]:
data_clas.show_batch()

In [0]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.freeze()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [0]:
learn.save('bio03_second')